
---

# 🧠 Playing with Multiple Models and Styles

---

## 🎯 Mission

In this lab, you will:

- Send a company name + website to different AI models
- Watch how **different models** (GPT, Claude, Gemini, Ollama) respond
- Control **how they behave** (funny, sincere, argumentative, or crazy)
- And see how **one simple input** can create **very different outputs** depending on the model and style!

---

## 🛠️ What’s Going On

| Step | What We’re Doing |
|:---|:---|
| 1 | Pick an AI model (GPT, Claude, Gemini, Ollama) |
| 2 | Pick a style (funny, sincere, crazy, argumentative) |
| 3 | AI reads the basic webpage info |
| 4 | AI writes a short company brochure — with your chosen style |
| 5 | Everything shows up nicely in a Gradio web app |

✅ It’s not about scraping — it's about **how AIs think and speak differently**!

---

## 🔥 Key Parts of the Code

| Part | What It Does |
|:---|:---|
| `Website` class | Grabs basic text (so AI has something to talk about) |
| `get_system_prompt(style)` | Tells the AI how to "act" |
| `invoke_*` functions | Talk to different models (GPT, Claude, Gemini, Ollama) |
| `gradio_app` | Launches a simple app you can actually click and play with |

---

## 🧠 Why This Matters

- **Multi-model thinking**: Not all AIs behave the same.  
- **Prompt engineering**: How you *ask* changes what you *get*.  
- **Controlled creativity**: You can guide AI tone and style — it's not just random.

✅ These are **foundational skills** for anything you’ll build in AI later — apps, agents, assistants, and beyond.

---

# 🎯 Final Thought

> **Today: Pick a model, pick a style.  
> Tomorrow: Build AI that fits your brand, your users, and your dreams.**

---


In [ ]:
# Setting up all the imports we need
# (a.k.a. assembling the Avengers for our app)
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr
import json

In [ ]:
# Load secret keys from the .env file
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Set up clients for OpenAI, Claude, and Gemini
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [ ]:
# Ollama is running locally (like your friendly neighborhood model)
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# 🏡 Scraping a website class
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')

        # Grab the title (or improvise if missing)
        self.title = soup.title.string if soup.title else "No title found"

        # Clean out junk we don't want: scripts, images, inputs
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        # Get readable page text
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [ ]:
# 🎨 Craft a system prompt based on style
def get_system_prompt(style: str) -> str:
    return f"""You are a {style} assistant that analyzes a company's landing page \
and creates a short brochure for customers, investors, and recruits. Respond in markdown."""

In [ ]:
# 🔥 OpenAI GPT - stream responses
def invoke_GPT(style: str, user_prompt):
    messages = [
        {"role": "system", "content": get_system_prompt(style)},
        {"role": "user", "content": user_prompt}
    ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [ ]:
# 🧊 Claude model - stream responses
def invoke_claude(style: str, user_prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=get_system_prompt(style),
        messages=[
            {"role": "user", "content": user_prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
# 🌟 Gemini model - a slightly fancier setup
def invoke_gemini(style: str, user_prompt: str):
    gemini_via_openai_client = OpenAI(
        api_key=google_api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    messages = [
        {"role": "system", "content": get_system_prompt(style)},
        {"role": "user", "content": user_prompt}
    ]
    response = gemini_via_openai_client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages
    )
    result = response.choices[0].message.content
    yield result


In [ ]:
# 🦙 Ollama (local llama model) - stream responses
def invoke_ollama(style: str, user_prompt: str):
    messages = [
        {"role": "system", "content": get_system_prompt(style)},
        {"role": "user", "content": user_prompt}
    ]
    response = requests.post(
        OLLAMA_API,
        headers=HEADERS,
        json={
            "model": MODEL,
            "messages": messages,
            "stream": True
        },
        stream=True
    )
    result = ""
    for chunk in response.iter_lines():
        if chunk:
            chunk_data = json.loads(chunk.decode('utf-8'))
            if 'message' in chunk_data and 'content' in chunk_data['message']:
                result += chunk_data['message']['content']
                yield result



In [ ]:
# 🎲 Model selector based on user choice
def model_case(model: str, style: str, user_prompt: str):
    model = model.lower()
    match model:
        case "gpt":
            return invoke_GPT(style, user_prompt)
        case "claude":
            return invoke_claude(style, user_prompt)
        case "gemini":
            return invoke_gemini(style, user_prompt)
        case "ollama":
            return invoke_ollama(style, user_prompt)
        case "_":
            pass  # silently fail if unknown


In [ ]:
# 🏗️ Building the final company brochure
def stream_brochure(company_name, url, model, style):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    yield from model_case(model, style, prompt)


In [ ]:
# 🎛️ Gradio app to tie everything together
def gradio_app():
    view = gr.Interface(
        fn=stream_brochure,
        inputs=[
            gr.Textbox(label="Company name:"),
            gr.Textbox(label="Landing page URL including http:// or https://"),
            gr.Dropdown(["GPT", "Claude", "Gemini", "Ollama"], label="Select model"),
            gr.Dropdown(["funny", "sincere", "argumentative", "crazy"], label="Select style")
        ],
        outputs=[gr.Markdown(label="Brochure:")],
        flagging_mode="never"
    )
    return view.launch()

In [ ]:

# 🏁 off to the races!
gradio_app()

---

# 🛠️ Mini Challenge: Add a New Style to Your Gradio App

---

## 🎯 Your Mission

Your Gradio app is already great —  
Now let's **level it up** even more!

✅ **Add one more style option** for the AI to use.

✅ **Add a new dropdown** choice in Gradio so users can pick this new setting.

---

## 🔥 What You Need to Do

1. **Pick a new style** for your assistant.  
   *(Example ideas: professional, sarcastic, poetic, motivational... up to you!)*

2. **Update the dropdown** list in the `gr.Interface` inside `gradio_app()`.

3. **No code changes needed** elsewhere!  
   (The system prompt builder will automatically handle your new style.)

---

## 🧠 Why This Matters

- This teaches you how to **extend your AI's personality**.  
- It shows how **small changes to prompts** create **big changes in outputs**.

✅ Building flexible, user-driven AIs is how real-world assistants work!

---

# 🚀 Final Hint

> **If you can control the style, you can control the entire vibe of your AI.**

---
